# Pablo the Thymio visits its first museum 

In [22]:
def convert_path_to_movements(path):
    for i in range (0, len(path)-1):
        p_x = path[i][0]
        p_y = path[i][1]
        n_x = path[i+1][0]
        n_y = path[i+1][1]
        theta = 0
        d = 0
        #This condition is needed, since when we do the Astar it will always use the start and end (visiting is start and end)
        if(n_x != p_x or n_y != p_y):
            #1st quadrant
            if(n_x > p_x and n_y >= p_y):
                theta = np.arctan(abs(n_y-p_y)/abs(n_x-p_x))
            #2nd quadrant
            elif(n_x <= p_x and n_y > p_y):
                theta = pi/2 + np.arctan(abs(n_x-p_x)/abs(n_y-p_y))
            #3rd quadrant
            elif(n_x < p_x and n_y <= p_y):
                theta = pi + np.arctan(abs(n_y-p_y)/abs(n_x-p_x))
            #4th quadrant
            elif(n_x >= p_x and n_y < p_y):
                theta = 3*pi/2 + np.arctan(abs(n_x-p_x)/abs(n_y-p_y))

            d = np.sqrt(np.power(abs(n_x-p_x),2)+np.power(abs(n_y-p_y),2))

            if(i == 0):
                move = np.array([[np.copy(theta)], [np.copy(d)]])
            else:
                move = np.append(move, [[np.copy(theta)],[np.copy(d)]], axis=1)
        else:
            move = np.append(move, [[0],[0]], axis=1)
    return move

In [44]:
def move_to_timer(theta,distance):
    #~ 0.0325 cm/s for 1 
    #1 carré = 6 cm
    #rotation: ~9.32s for 360° with [100, -100] motor inputs => ~ 0.0067 rad/s for [1, -1] motor inputs
    distance_in_1_s = 0.0325
    time_for_one_turn = 9.32
    distance_cm = distance*6
    
    time_to_turn = theta*time_for_one_turn/(2*pi)
    time_forward =  distance_cm/distance_in_1_s

    return time_to_turn, time_forward
#On va utiliser ces timer dans la fonction pour avancer

In [47]:
print(move_to_timer(3.12,5))

(4.627971097203369, 923.0769230769231)


In [48]:
State = 0 # State == 0 => Thymio goes straight, State == 1 => Thymio is in obstacle avoidance mode
Velocity_0 = 100
Obstacle_gain = [10, 15]
Threshold_enter_obstavoid = 20 # Threshold to enter obstacle avoidance mode
Threshold_quit_obstavoid = 10 # Threshold to quit obstacle avoidance mode
Timer = 1 * 10**(-3) # 1ms
current_move = move.remove(move[0])
d_needed = current_move[0]
theta_needed = current_move[1]
d_parcourue = 0

while(1 and x+epsilon < path[-1][0] and y+epsilon < path[-1][1]):
    if (State == 0 and
        (th["prox.horizontal"][0] > Threshold_enter_obstavoid or
        th["prox.horizontal"][1] > Threshold_enter_obstavoid or
        th["prox.horizontal"][2] > Threshold_enter_obstavoid or
        th["prox.horizontal"][3] > Threshold_enter_obstavoid or
        th["prox.horizontal"][4] > Threshold_enter_obstavoid)): # enter obstacle avoidance
        State = 1
    elif (State == 1 and
        th["prox.horizontal"][0] < Threshold_quit_obstavoid and
        th["prox.horizontal"][1] < Threshold_enter_obstavoid and
        th["prox.horizontal"][2] < Threshold_enter_obstavoid and
        th["prox.horizontal"][3] < Threshold_enter_obstavoid and
        th["prox.horizontal"][4] < Threshold_quit_obstavoid): # quit obstacle avoidance
        State = 0
    
    if State == 0: #not avoidance, we do whatever
        #Correction orientation en temps réel
        if theta+epsilon_theta < theta_needed and theta-epsilon_theta > theta_needed :
            if(theta+epsilon_theta<theta_needed):
                th.set_var("motor.right.target", 100)
                th.set_var("motor.left.target", 2**16 - 100)
            else:
                th.set_var("motor.right.target", 2**16 - 100)
                th.set_var("motor.left.target", 100)
        #Sinon avance chillement
        else:
            th.set_var("motor.right.target", 100)
            th.set_var("motor.left.target", 100)
            d_parcourue += d_main
            if(d_parcourue >= d_needed):
                #On vide la liste au fur et à mesure qu'on fait les déplacements
                current_move = move.remove(move[0])
                d_needed = current_move[0]
                theta_needed = current_move[1]
                d_parcourue = 0
    else: # avoid obstacle depending on sensors values
        if (th["prox.horizontal"][0] == 0 and
            th["prox.horizontal"][1] == 0 and
            th["prox.horizontal"][3] == 0 and
            th["prox.horizontal"][4] == 0):
            Left_gain = 0
            Right_gain = int(Obstacle_gain[1]*(th["prox.horizontal"][2]/100))
        else:
            Left_gain = int(Obstacle_gain[0]*(th["prox.horizontal"][4]/100)) + int(Obstacle_gain[1]*(th["prox.horizontal"][3]/100))
            Right_gain = int(Obstacle_gain[0]*(th["prox.horizontal"][0]/100)) + int(Obstacle_gain[1]*(th["prox.horizontal"][1]/100))
        
        if Left_gain > Velocity_0:
            th.set_var("motor.left.target", 2**16 + (Velocity_0 - Left_gain))
        else:
            th.set_var("motor.left.target", (Velocity_0 - Left_gain))
        
        if Right_gain > Velocity_0:
            th.set_var("motor.right.target", 2**16 + (Velocity_0 - Right_gain))
        else:
            th.set_var("motor.right.target", (Velocity_0 - Right_gain))
        
    time.sleep(Timer) # wait 1ms before relooping


th.set_var("motor.right.target", 0)
th.set_var("motor.left.target", 0)

TypeError: can't multiply sequence by non-int of type 'float'